In [1]:
using CSV
using DataFrames
using JSON
using ArchGDAL
using Proj
using Rasters
using Base.Threads
using JLD2
using Lux
using LuxCore
using EasyHybrid
using Optimisers
using Statistics
using Plots
include("helpers.jl")
using .Helpers
version = "v20251219"

"v20251219"

## Overlay

In [2]:
# load cov list
cov = JSON.parsefile("./cov_path_full.json");
pnames = collect(keys(cov))
paths = collect(values(cov))

res_m = 100 # meters
bboxmine = (8.956051,51.815757,10.450192,53.154421) # examine area in northern DE, suggested by Bernhard
bbox3035 = convert_bbox_wgs84_to_3035(bboxmine);
xs, ys = make_grid_3035(bbox3035, res_m);

tf = Proj.Transformation("EPSG:3035", "EPSG:4326") # because it's always 4326, so we do it in lazy way

Transformation unknown
    source: ETRS89-extended / LAEA Europe
    target: WGS 84
    direction: forward


In [3]:
tnames = pnames#[10:30]
tpaths = paths#[10:30]  
println("Julia threads: ", Threads.nthreads())
println("length: ", length(tpaths))

Julia threads: 96
length: 362


In [ ]:
chunk_size = 40
out = Vector{Any}(undef, length(tpaths))
m = Int[]
lock_m = ReentrantLock()
iii = 0
for chunk in Iterators.partition(eachindex(tpaths), chunk_size)
    println("chunk - $(iii)")

    @time @threads for i in chunk
        try
            out[i] = sample_tiff_onto_grid(tpaths[i], xs, ys, tf)
        catch
            lock(lock_m) do
                push!(m, i)
            end
        end
    end
    
    iii = iii + 1
end

chunk - 0
 83.943894 seconds (321.11 M allocations: 9.654 GiB, 2.31% gc time, 68.47% compilation time)
chunk - 1
 83.502818 seconds (331.53 M allocations: 9.762 GiB, 2.35% gc time, 0.50% compilation time)
chunk - 2


In [ ]:
df = DataFrame()
nx = length(xs)
ny = length(ys)
df.x3035 = repeat(xs, inner=ny)
df.y3035 = repeat(ys, outer=nx)
for i in eachindex(tnames)
    df[!, Symbol(tnames[i])] = out[i]
end

In [ ]:
CSV.write("overlaid_$(res_m)m_$(version).csv", df)

## prepare the data as how it's processed before training

In [ ]:
using CSV, DataFrames
# prepare data
# load in preprocessed data to get predictors
datafile = "/mnt/tupi/HybridModeling/EasyDensity.jl/data/lucas_preprocessed_v20251125.csv"
oridf = CSV.read(datafile, DataFrame; normalizenames=true)
predictors = Symbol.(names(oridf))[18:end-6]; # CHECK EVERY TIME 

In [ ]:
using Statistics
# ? move the `csv` file into the `BulkDSOC/data` folder (create folder)
df_o = CSV.read("/mnt/tupi/HybridModeling/EasyDensity.jl/data/lucas_overlaid.csv", DataFrame, normalizenames=true);
println(size(df_o));

############################
###### clean targets #######
############################

# filter horizon depth = 10 cm
df_o = df_o[df_o.hzn_dep .== 10, :];
select!(df_o, Not(:hzn_dep));
println(size(df_o))

# identify noise time supervise
gdf = groupby(df_o, :id);
df_o.maxdiff = fill(0.0, nrow(df_o));  # initialize noise column
# compute max abs difference of SOCconc per id
for sub in groupby(df_o, :id)
    soc = sort(sub.soc)

    if length(soc) < 2
        maxdiff = -1
    else
        maxdiff = maximum(abs.(diff(soc)))
    end

    df_o[df_o.id .== sub.id[1], :maxdiff] .= maxdiff
    
end
println(size(df_o))
df_o = df_o[df_o.maxdiff .<= 50, :];
println(size(df_o))

# coords = collect(zip(df_o.lat, df_o.lon));

########################
###### clean cov #######
########################
# t clean covariates
names_cov = Symbol.(names(df_o))[18:end-1];

# Fix soilsuite and cropland extent columns
for col in names_cov
    if occursin("_soilsuite_", String(col))
        df_o[!, col] = replace(df_o[!, col], missing => 0)
    elseif occursin("cropland_extent_", String(col))
        df_o[!, col] = replace(df_o[!, col], missing => 0)
        df_o[!, col] .= ifelse.(df_o[!, col] .> 0, 1, 0)
    end
end

# rm missing values: 1. >5%, drop col; 2. <=5%, drop row
cols_to_drop_row = Symbol[];
cols_to_drop_col = Symbol[];
for col in names_cov
    n_missing = count(ismissing, df_o[!, col])
    frac_missing = n_missing / nrow(df_o)
    if frac_missing > 0.05
        println(n_missing, " ", col)
        select!(df_o, Not(col))  # drop the column
        push!(cols_to_drop_col, col)  
    elseif n_missing > 0
        # println(n_missing, " ", col)
        push!(cols_to_drop_row, col)  # collect column name
    end

    if occursin("CHELSA_kg", String(col)) 
        push!(cols_to_drop_col, col) 
        select!(df_o, Not(col))  # rm kg catagorical col
    end 
end

names_cov = filter(x -> !(x in cols_to_drop_col), names_cov) # remove cols-to-drop from names_cov
if !isempty(cols_to_drop_row) 
    df_o = subset(df_o, cols_to_drop_row .=> ByRow(!ismissing)) # drop rows with missing values in cols_to_drop_row
end
println(size(df_o))

cols_to_drop_col = Symbol[] 
for col in names_cov
    if std(df_o[:,col])==0
        push!(cols_to_drop_col, col)  # rm constant col (std==0)
        select!(df_o, Not(col))
    end
end
names_cov = filter(x -> !(x in cols_to_drop_col), names_cov) # remove cols-to-drop from names_cov
println(size(df_o))

# for col in names_cov # to check covairate distribution
#     println(string(col)[1:10], ' ', round(std(df[:, col]); digits=2), ' ', round(mean(df[:, col]); digits=2))
# end

# # Normalize covariates by (x-mean) / std
means = map(c -> mean(skipmissing(df_o[!, c])), predictors)
stds  = map(c -> std(skipmissing(df_o[!, c])), predictors)


In [ ]:
## apply the normalizations to new training data
# get the overlaid data
version = "v20251219"
df = CSV.read("overlaid_$(version).csv", DataFrame)

# mend crop and soil suite layers
for col in predictors
    if occursin("_soilsuite_", String(col))
        df[!, col] = replace(df[!, col], missing => 0)
    elseif occursin("cropland_extent_", String(col))
        df[!, col] = replace(df[!, col], missing => 0)
        df[!, col] .= ifelse.(df[!, col] .> 0, 1, 0)
    end
end

for (i, col) in enumerate(predictors)
    df[!, col] = (Float64.(df[!, col]) .- means[i]) ./ stds[i]
end


In [ ]:
clean(x) = filter(!isnan, skipmissing(x))

rows = Vector{NamedTuple}()

for col in predictors
    v_o = clean(oridf[!, col])
    v_d = clean(df[!, col])

    push!(rows, (
        variable   = String(col),
        q05_oridf  = quantile(v_o, 0.05),
        q05_df     = quantile(v_d, 0.05),
        q50_oridf  = quantile(v_o, 0.50),
        q50_df     = quantile(v_d, 0.50),
        q95_oridf  = quantile(v_o, 0.95),
        q95_df     = quantile(v_d, 0.95)
    ))
end

qt = DataFrame(rows)
CSV.write("predictor_quantiles_check.csv", qt)


In [ ]:
CSV.write("production_preprocessed_$(res_m)m_$(version).csv", df)

## load model, retrain on all data and predict

In [ ]:
# jldfile = "/mnt/tupi/HybridModeling/EasyDensity.jl/model/best_model_03_hybridNN_config105_fold4.jld2"
# hm = load_last_epoch(jldfile);
# ps = hm[1]
# st = hm[2]

# prepare data
# load in preprocessed data to get predictors
datafile = "/mnt/tupi/HybridModeling/EasyDensity.jl/data/lucas_preprocessed_v20251125.csv"
oridf = CSV.read(datafile, DataFrame; normalizenames=true)
predictors = Symbol.(names(oridf))[18:end-6]; # CHECK EVERY TIME 

# parameters
scalers = Dict(
    :SOCconc   => 0.151, # g/kg, log(x+1)*0.151
    :CF        => 0.263, # percent, log(x+1)*0.263
    :BD        => 0.529, # g/cm3, x*0.529
    :SOCdensity => 0.167, # kg/m3, log(x)*0.167
);

parameters = (
    SOCconc = (0.01f0, 0.0f0, 1.0f0),   # fraction
    CF      = (0.15f0, 0.0f0, 1.0f0),   # fraction,
    oBD     = (0.20f0, 0.05f0, 0.40f0),  # also NN learnt, g/cm3
    mBD     = (1.20f0, 0.75f0, 2.0f0),  # NN leanrt
)
neural_param_names = [:SOCconc, :CF, :mBD, :oBD]
forcing = Symbol[]
targets = [:BD, :SOCconc, :SOCdensity, :CF]   

hmb = constructHybridModel(
    predictors,
    forcing,
    targets,
    SOCD_model,
    parameters,
    neural_param_names,
    [];
    hidden_layers = [256, 128, 64, 32],
    activation = gelu,
    scale_nn_outputs = true,
    input_batchnorm = false,
    start_from_default = true
);

rlt = train(
    hmb, oridf, ();
    nepochs = 200,
    batchsize = bs,
    opt = AdamW(lr),
    training_loss = :mse,
    loss_types = [:mse, :r2],
    shuffleobs = true,
    file_name = "prod_SiNN.jld2",
    random_seed = 42,
    patience = 15,
    yscale = identity,
    monitor_names = [:oBD, :mBD],
    agg = mean,
    return_model = :best,
    show_progress = false,
    plotting = false,
    hybrid_name = "prod_SiNN" 
)


In [ ]:
# get the preprocessed overlaid data
df = CSV.read("production_preprocessed_$(res_m)m_$(version).csv", DataFrame)
x_test = to_keyedArray(Float32.(df[!, predictors]));

ps, st = rlt.ps, rlt.st
ŷ_test, st_test = best_model(x_test, ps, LuxCore.testmode(st))

# prediction
for var in [:BD, :SOCconc, :CF, :SOCdensity, :oBD, :mBD]
    if hasproperty(ŷ_test, var)
        val = getproperty(ŷ_test, var)

        if val isa AbstractVector && length(val) == nrow(df)
            df[!, Symbol("pred_", var)] = val # per row

        elseif (val isa Number) || (val isa AbstractVector && length(val) == 1)
            df[!, Symbol("pred_", var)] = fill(Float32(val isa AbstractVector ? first(val) : val), nrow(df))
        end
    end
end

# inverse SOC concentration (g/kg)
df[!, :soc] = exp.(df[!, :pred_SOCconc] ./ scalers[:SOCconc]) .- 1;
# inverse CF (fraction, NOT percent)
df[!, :cf] = (exp.(df[!, :pred_CF] ./ scalers[:CF]) .- 1) ./ 100;
# inverse bulk density (g/cm3)
df[!, :bd] = df[!, :pred_BD] ./ scalers[:BD];
# inverse SOC density (kg/m3)
df[!, :ocd] = exp.(df[!, :pred_SOCdensity] ./ scalers[:SOCdensity]);

println(size(df))
save_col = [
    :x3035, :y3035,
    :pred_SOCconc, :pred_CF, :pred_BD,
    :pred_SOCdensity,
    :soc, :ocd, :bd, :cf
]

CSV.write("out_$(res_m)m_$(version).csv", df[:, save_col])


## check predictions

In [ ]:
using Statistics
using Plots
for col in ["pred_BD", "pred_SOCconc", "pred_CF", "pred_SOCdensity"]

    vals = df[:, col]

    # 有效值（非 missing 且非 NaN）
    valid_vals = filter(x -> !ismissing(x) && !isnan(x), vals)

    n_valid = length(valid_vals)
    vmin = minimum(valid_vals)
    vmax = maximum(valid_vals)

    println("Variable: $col")
    println("  Valid count = $n_valid")
    println("  Min = $vmin")
    println("  Max = $vmax\n")

    histogram(
        vals;
        bins = 50,
        xlabel = col,
        ylabel = "Frequency",
        title = "Histogram of $col",
        lw = 1,
        legend = false
    )
    display(current())
end

## save as tiff

In [ ]:
for var in [:bd, :soc, :cf, :ocd, :pred_oBD, :pred_mBD]
    write_geotiff_from_grid(
        df,
        xs,
        ys,
        var,
        "pred_$(var).tif"
    )
end